In [64]:
import copy

In [88]:
class Coordinate:
    '''
    Just a object to store coordinates. Nothing fancy
    '''
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

class Moon:
    '''
    An object containing each moon
    '''
    def __init__(self, name, x, y, z):
        self.name = name
        self.pos = [Coordinate(int(x), int(y), int(z))]
        self.vel = [Coordinate(0, 0, 0)]

    def getPositionAtTime(self, time):
        '''
        returns the moon's position (coordinates) at a given time
        '''
        try:
            return self.pos[time]
        except:
            return None
    
    def getVelocityAtTime(self, time):
        '''
        returns a moon's velocity at a given time.
        '''
        try:
            return self.vel[time]
        except:
            return None
        
    def initTimeStep(self, time_step):
        '''
        initialises the time step provided to be the same as the step before it initially
        '''
        if len(self.pos) <= time_step:
            newpos = copy.deepcopy(self.pos[time_step-1])
            self.pos.append(newpos)
        if len(self.vel) <= time_step:
            newvel = copy.deepcopy(self.vel[time_step-1])
            self.vel.append(newvel)
    
    def getPositionString(self, time_step):
        '''
        Returns the current position of this moon to a string like on the puzzle's page
        '''
        return "x={:3}, y={:3}, z={:3}".format(self.pos[time_step].x, self.pos[time_step].y, self.pos[time_step].z)
    
    def getVelocityString(self, time_step):
        '''
        Returns the current velocity of this moon to a string like on the puzzle's page
        '''
        return "x={:3}, y={:3}, z={:3}".format(self.vel[time_step].x, self.vel[time_step].y, self.vel[time_step].z)
    
    def applyVelocity(self, time_step):
        '''
        Once we calculate the velocity of this position, we need to apply it to the position to update this step
        '''
        self.pos[time_step].x += self.vel[time_step].x
        self.pos[time_step].y += self.vel[time_step].y
        self.pos[time_step].z += self.vel[time_step].z
    
    def getPotentialEnergy(self, time_step):
        '''
        Calculates the potential energy in the moon for a given time step
        '''
        pot = abs(self.pos[time_step].x) + abs(self.pos[time_step].y) + abs(self.pos[time_step].z)
        return pot 
    
    def getKineticEnergy(self, time_step):
        '''
        Calculate the kinetic energy in the moon for a given time step
        '''
        pot = abs(self.vel[time_step].x) + abs(self.vel[time_step].y) + abs(self.vel[time_step].z)
        return pot
        
class System:
    '''
    This will store the information for the entire planetary system
    '''
    def __init__(self, moon_names, scan):
        self.moons = list()
        for idx, pos in enumerate(scan):
            pos_list = pos.replace("<","").replace(">","").replace("x=","").replace("y=","").replace("z=","").split(",")
            self.moons.append(Moon(moon_names[idx], pos_list[0], pos_list[1], pos_list[2]))
        
    def calculateVelocity(self, time_step):
        '''
        This calculates the velocity for the given time step based upon the differences in position.
        I probably could do this more elegantly, but this works for now...
        '''
        for i in range(len(self.moons)):
            self.moons[i].initTimeStep(time_step)
            for j in range(i + 1, len(self.moons)):
                self.moons[j].initTimeStep(time_step)
                if self.moons[i].pos[time_step].x > self.moons[j].pos[time_step].x:
                    self.moons[i].vel[time_step].x -= 1
                    self.moons[j].vel[time_step].x += 1
                elif self.moons[i].pos[time_step].x < self.moons[j].pos[time_step].x:
                    self.moons[i].vel[time_step].x += 1
                    self.moons[j].vel[time_step].x -= 1
                    
                if self.moons[i].pos[time_step].y > self.moons[j].pos[time_step].y:
                    self.moons[i].vel[time_step].y -= 1
                    self.moons[j].vel[time_step].y += 1
                elif self.moons[i].pos[time_step].y < self.moons[j].pos[time_step].y:
                    self.moons[i].vel[time_step].y += 1
                    self.moons[j].vel[time_step].y -= 1
                    
                if self.moons[i].pos[time_step].z > self.moons[j].pos[time_step].z:
                    self.moons[i].vel[time_step].z -= 1
                    self.moons[j].vel[time_step].z += 1
                elif self.moons[i].pos[time_step].z < self.moons[j].pos[time_step].z:
                    self.moons[i].vel[time_step].z += 1
                    self.moons[j].vel[time_step].z -= 1
        
        for moon in self.moons:
            moon.applyVelocity(time_step)
                    
    def getPosition(self, time_step):
        '''
        This will print out the position of the moons at a given time step
        '''
        print("After {} steps:".format(time_step))
        for moon in self.moons:
            if len(moon.name) > 6:
                print("{}:\t pos=<{}>, \t vel=<{}>".format(moon.name, moon.getPositionString(time_step), moon.getVelocityString(time_step)))
            else:
                print("{}:\t\t pos=<{}>, \t vel=<{}>".format(moon.name, moon.getPositionString(time_step), moon.getVelocityString(time_step)))
                    
    def getEnergy(self, time_step, print_step):
        '''
        This will calculate the energy for a given time step and print out the 
        results only if this is a printing step
        '''
        if time_step % print_step == 0:
            print("Energy after {} steps:".format(time_step))
        total = 0
        for moon in self.moons:
            pot = moon.getPotentialEnergy(time_step)
            kin = moon.getKineticEnergy(time_step)
            moon_total = kin * pot
            total += moon_total
            
            if time_step % print_step == 0:
                pot_str = "{:3} + {:3} + {:3} = {:3}".format(moon.pos[time_step].x, moon.pos[time_step].y, moon.pos[time_step].z, pot)
                kin_str = "{:3} + {:3} + {:3} = {:3}".format(moon.vel[time_step].x, moon.vel[time_step].y, moon.vel[time_step].z, kin)
                total_str = "{:3} * {:3} = {:4}".format(pot, kin, moon_total)

                if len(moon.name) > 6:
                    print("{}:\t pos={};\t vel={};\t total={}".format(moon.name, pot_str, kin_str, total_str))
                else:
                    print("{}:\t\t pos={};\t vel={};\t total={}".format(moon.name, pot_str, kin_str, total_str))

        if time_step % print_step == 0:
            print("Sum of total energy: {}".format(total))
            
    def timeShift(self, num_steps, print_step):
        '''
        This is the main controlling function that will loop over a given range of steps and make calculations
        '''
        jupiter.getPosition(0)
        for i in range(1, num_steps):
            jupiter.calculateVelocity(i)
            if i % print_step == 0:
                jupiter.getPosition(i)
            jupiter.getEnergy(i, print_step)

In [89]:
'''
Test 1
'''
moon_list = ["Io", "Europa", "Ganymede", "Callisto"]

test_scan = '''<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>'''

scan = test_scan.split("\n")
jupiter = System(moon_list, scan)

jupiter.timeShift(11, 1)

After 0 steps:
Io:		 pos=<x= -1, y=  0, z=  2>, 	 vel=<x=  0, y=  0, z=  0>
Europa:		 pos=<x=  2, y=-10, z= -7>, 	 vel=<x=  0, y=  0, z=  0>
Ganymede:	 pos=<x=  4, y= -8, z=  8>, 	 vel=<x=  0, y=  0, z=  0>
Callisto:	 pos=<x=  3, y=  5, z= -1>, 	 vel=<x=  0, y=  0, z=  0>
After 1 steps:
Io:		 pos=<x=  2, y= -1, z=  1>, 	 vel=<x=  3, y= -1, z= -1>
Europa:		 pos=<x=  3, y= -7, z= -4>, 	 vel=<x=  1, y=  3, z=  3>
Ganymede:	 pos=<x=  1, y= -7, z=  5>, 	 vel=<x= -3, y=  1, z= -3>
Callisto:	 pos=<x=  2, y=  2, z=  0>, 	 vel=<x= -1, y= -3, z=  1>
Energy after 1 steps:
Io:		 pos=  2 +  -1 +   1 =   4;	 vel=  3 +  -1 +  -1 =   5;	 total=  4 *   5 =   20
Europa:		 pos=  3 +  -7 +  -4 =  14;	 vel=  1 +   3 +   3 =   7;	 total= 14 *   7 =   98
Ganymede:	 pos=  1 +  -7 +   5 =  13;	 vel= -3 +   1 +  -3 =   7;	 total= 13 *   7 =   91
Callisto:	 pos=  2 +   2 +   0 =   4;	 vel= -1 +  -3 +   1 =   5;	 total=  4 *   5 =   20
Sum of total energy: 229
After 2 steps:
Io:		 pos=<x=  5, y= -3, z= -1>, 	 vel

In [91]:
'''
Test 2
'''
test_scan = '''<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>'''

scan = test_scan.split("\n")
jupiter = System(moon_list, scan)

jupiter.timeShift(101, 10)

After 0 steps:
Io:		 pos=<x= -8, y=-10, z=  0>, 	 vel=<x=  0, y=  0, z=  0>
Europa:		 pos=<x=  5, y=  5, z= 10>, 	 vel=<x=  0, y=  0, z=  0>
Ganymede:	 pos=<x=  2, y= -7, z=  3>, 	 vel=<x=  0, y=  0, z=  0>
Callisto:	 pos=<x=  9, y= -8, z= -3>, 	 vel=<x=  0, y=  0, z=  0>
After 10 steps:
Io:		 pos=<x= -9, y=-10, z=  1>, 	 vel=<x= -2, y= -2, z= -1>
Europa:		 pos=<x=  4, y= 10, z=  9>, 	 vel=<x= -3, y=  7, z= -2>
Ganymede:	 pos=<x=  8, y=-10, z= -3>, 	 vel=<x=  5, y= -1, z= -2>
Callisto:	 pos=<x=  5, y=-10, z=  3>, 	 vel=<x=  0, y= -4, z=  5>
Energy after 10 steps:
Io:		 pos= -9 + -10 +   1 =  20;	 vel= -2 +  -2 +  -1 =   5;	 total= 20 *   5 =  100
Europa:		 pos=  4 +  10 +   9 =  23;	 vel= -3 +   7 +  -2 =  12;	 total= 23 *  12 =  276
Ganymede:	 pos=  8 + -10 +  -3 =  21;	 vel=  5 +  -1 +  -2 =   8;	 total= 21 *   8 =  168
Callisto:	 pos=  5 + -10 +   3 =  18;	 vel=  0 +  -4 +   5 =   9;	 total= 18 *   9 =  162
Sum of total energy: 706
After 20 steps:
Io:		 pos=<x=-10, y=  3, z= -4>, 	 

In [92]:
'''
Real Run
'''

scan_string = '''<x=0, y=4, z=0>
<x=-10, y=-6, z=-14>
<x=9, y=-16, z=-3>
<x=6, y=-1, z=2>'''

scan = scan_string.split("\n")
jupiter = System(moon_list, scan)

# Because we're doing so many steps, let's only print out every 100 to make sure we're on track.
jupiter.timeShift(1001, 100)

After 0 steps:
Io:		 pos=<x=  0, y=  4, z=  0>, 	 vel=<x=  0, y=  0, z=  0>
Europa:		 pos=<x=-10, y= -6, z=-14>, 	 vel=<x=  0, y=  0, z=  0>
Ganymede:	 pos=<x=  9, y=-16, z= -3>, 	 vel=<x=  0, y=  0, z=  0>
Callisto:	 pos=<x=  6, y= -1, z=  2>, 	 vel=<x=  0, y=  0, z=  0>
After 100 steps:
Io:		 pos=<x= -4, y=  4, z=  5>, 	 vel=<x=  6, y= -4, z=  0>
Europa:		 pos=<x=-12, y=  1, z=  0>, 	 vel=<x= -8, y=  4, z= -1>
Ganymede:	 pos=<x=  6, y= -7, z= -2>, 	 vel=<x=  2, y=  3, z=  5>
Callisto:	 pos=<x= 15, y=-17, z=-18>, 	 vel=<x=  0, y= -3, z= -4>
Energy after 100 steps:
Io:		 pos= -4 +   4 +   5 =  13;	 vel=  6 +  -4 +   0 =  10;	 total= 13 *  10 =  130
Europa:		 pos=-12 +   1 +   0 =  13;	 vel= -8 +   4 +  -1 =  13;	 total= 13 *  13 =  169
Ganymede:	 pos=  6 +  -7 +  -2 =  15;	 vel=  2 +   3 +   5 =  10;	 total= 15 *  10 =  150
Callisto:	 pos= 15 + -17 + -18 =  50;	 vel=  0 +  -3 +  -4 =   7;	 total= 50 *   7 =  350
Sum of total energy: 799
After 200 steps:
Io:		 pos=<x=  9, y=  5, z= -7>,